In [1]:
from autogen import ConversableAgent, register_function
#from rag_tools import * # Imports all the functions

In [6]:
#TODO: Add this part in rag_tools
from neo4j import GraphDatabase as gd

d = gd.driver("bolt://localhost:7687" , auth=None)
d.verify_connectivity()
s = d.session()

# Test if nodes & relationships are correctly loaded
res = s.run("""match (m:Movie) return m.title as title limit 5""").data()
print(res)

def get_movie_plot(movie_title: str) -> str:
    plot = s.run("""match (m:Movie {title: ${movie_title}}) return m.plot""").data()

    return plot

[{'title': 'Toy Story'}, {'title': 'Jumanji'}, {'title': 'Grumpier Old Men'}, {'title': 'Waiting to Exhale'}, {'title': 'Father of the Bride Part II'}]


In [7]:
#TODO: future config_file should be a json file to not repeat this lines
config_list = [{
        "model": "llama3.1",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama"
}]

prompt = """You are a movie expert providing information about movies.
Be as helpful as possible and return as much information as possible.
Do not answer any questions that do not relate to movies or actors.

Do not answer any questions using your pre-trained knowledge, only use the information provided in the context.
You may decide to use some external tools if necessary"
If you are happy with the result of the tool, just say TERMINATE
"""

In [8]:
graph_tool = ConversableAgent(
    "graph_tool",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    # is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

graph_driver = ConversableAgent(
    "graph_driver",
    system_message = prompt,
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

register_function(
    get_movie_plot,
    caller = graph_driver,
    executor = graph_tool,
    description = "get the plot of a movie from a graph database. this service only receive the formal name of a movie"
)

graph_tool.initiate_chat(
    graph_driver,
    message = "Can you find a movie that talks about toys coming to life?",
    max_turns = 2
)

graph_tool (to graph_driver):

Can you find a movie that talks about toys coming to life?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-02 09:29:27] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
graph_driver (to graph_tool):


***** Suggested tool call (call_mlux57ge): get_movie_plot *****
Arguments: 
{"movie_title":"Toy Story"}
***************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION get_movie_plot...
graph_tool (to graph_driver):

graph_tool (to graph_driver):

***** Response from calling tool (call_mlux57ge) *****
Error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '{': expected an identi

ChatResult(chat_id=None, chat_history=[{'content': 'Can you find a movie that talks about toys coming to life?', 'role': 'assistant', 'name': 'graph_tool'}, {'content': '', 'tool_calls': [{'id': 'call_mlux57ge', 'function': {'arguments': '{"movie_title":"Toy Story"}', 'name': 'get_movie_plot'}, 'type': 'function'}], 'role': 'assistant'}, {'content': 'Error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'{\': expected an identifier or an integer value (line 1, column 25 (offset: 24))\n"match (m:Movie {title: ${movie_title}}) return m.plot"\n                         ^}', 'tool_responses': [{'tool_call_id': 'call_mlux57ge', 'role': 'tool', 'content': 'Error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'{\': expected an identifier or an integer value (line 1, column 25 (offset: 24))\n"match (m:Movie {title: ${movie_title}}) return m.plot"\n                         ^}'}], 'role': 'tool', 'name': 'graph_tool'}, {'content': "It seems like I 